In [ ]:
!pip install networkx

##Методы для работы с графами:
 - nx.Graph(): Создает пустой граф.
 - G.add_node(node): Добавляет одну вершину в граф.
 - G.add_nodes_from(nodes): Добавляет несколько вершин в граф.
 - G.add_edge(u, v): Добавляет ребро между вершинами u и v.
 - G.add_edges_from(edges): Добавляет несколько ребер в граф.
 - G.nodes(): Возвращает список всех вершин в графе.
 - G.edges(): Возвращает список всех ребер в графе.
 - G.degree(node): Возвращает степень вершины (количество ребер, соединенных с вершиной).
 - G.number_of_nodes(): Возвращает количество вершин в графе.
 - G.number_of_edges(): Возвращает количество ребер в графе.
 - G.neighbors(node): Возвращает список вершин, соединенных с данной вершиной.
 - G.adjacency(): Возвращает словарь, где ключи - вершины, а значения - списки вершин, соединенных с ключом.
 - G.subgraph(nodes): Возвращает подграф, содержащий только указанные вершины.
 - G.copy(): Возвращает копию графа.
##Методы для работы с вершинами:
 - G.nodes(data=True): Возвращает список вершин с данными.
 - G.nodes.data(): Возвращает итератор, который генерирует пары (узел, данные).
 - G.nodes.get(node): Возвращает данные вершины.
 - G.nodes.set(node, **attr): Устанавливает данные вершины.
 - G.nodes.remove(node): Удаляет вершину из графа.
##Методы для работы с ребрами:
 - G.edges(data=True): Возвращает список ребер с данными.
 - G.edges.data(): Возвращает итератор, который генерирует пары (ребро, данные).
 - G.edges.get(edge): Возвращает данные ребра.
 - G.edges.set(edge, **attr): Устанавливает данные ребра.
 - G.edges.remove(edge): Удаляет ребро из графа.

_____________________________________________________________________

##Список дел

число вершин - ✔

число ребер - ✔

диаметр графа - ✔

радиус графа - ✔

число компонент связности - X

цикломатическое число - X

среднее значение степеней вершин - ✔

максимальная степень вершины - ✔

минимальная степень вершины - ✔

медиана значений степеней вершин - ✔

мода степеней вершин - ✔

хроматическое число - ✔ (Требует проверки)

планарность - X

Для планарных графов:

число граней - X

среднее значение числа рёбер в грани - X

медианное значение - X

максимальное и минимальное значение - X

_____________________________________________________________________

Импорт библиотек

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random

Функция для генерации графа

In [ ]:
def generate_random_graph(num_nodes, edge_probability):
    """
    Генерирует случайный граф с заданным количеством узлов и вероятностью создания ребра.

    :param num_nodes: Количество узлов в графе.
    :param edge_probability: Вероятность создания ребра между двумя узлами.
    :return: Сгенерированный граф.
    """
    G = nx.Graph()
    # Добавляем узлы в граф
    G.add_nodes_from(range(num_nodes))

    # Создаем ребра с учетом вероятности
    for i in range(num_nodes):
        for j in range(i+1, num_nodes):
            if random.random() < edge_probability:
                G.add_edge(i, j)

    return G

Сохранение графа в виде графика

In [ ]:
def save_graph(G, filename):
    """
    Сохраняет граф G в файл с заданным именем.

    :param G: Граф для сохранения.
    :param filename: Имя файла для сохранения графа.
    """
    plt.figure(figsize=(10, 10))
    nx.draw(G, with_labels=True)
    plt.savefig(filename, dpi=300)
    plt.close()

Вычисление диаметра и радиуса

In [ ]:
def calculate_diameter_and_radius(G):
    """
    Вычисляет диаметр и радиус графа.

    Диаметр графа определяется как максимальное расстояние между двумя вершинами графа.
    Радиус графа определяется как минимальное из максимальных расстояний от каждой вершины до всех остальных вершин.

    :param G: Граф, для которого вычисляются диаметр и радиус.
    :return: Кортеж, содержащий диаметр и радиус графа.
    """
    diameter = nx.diameter(G)
    radius = nx.radius(G)
    return diameter, radius

Число компонент связности

In [ ]:
def number_connected_components(G):
  """
  Вычисляет число компонент связности графа G.
  """
  return nx.number_connected_components(G)

Цикломатическое число

Значение степеней вершин

In [ ]:
from collections import Counter
def meaning_of_degrees(G):
    """
    Вычисляет максимальное, минимальное, среднее, медианное значение и моду степеней вершин в графе.

    Степень вершины определяется как количество ребер, соединенных с этой вершиной.

    :param G: Граф, для которого вычисляются значения.
    :return: Кортеж, содержащий максимальное, минимальное, среднее и медианное значение степеней вершин.
    """
    degrees = [val for (node, val) in G.degree()]
    max_value = max(degrees)
    min_value = min(degrees)
    mean_value = sum(degrees) / len(degrees)
    median_value = sorted(degrees)[len(degrees) // 2]
    degree_counts = Counter(degrees)
    mode_value = degree_counts.most_common(1)[0][0]

    return max_value, min_value, mean_value, median_value, mode_value

Хроматическое число

In [ ]:
def is_safe(vertex, color, color_map, G):
    """
    Проверяет, можно ли безопасно окрасить вершину в указанный цвет.

    :param vertex: Индекс вершины.
    :param color: Цвет для окраски.
    :param color_map: Текущее отображение вершин на цвета.
    :param G: Граф.
    :return: True, если окраска безопасна, иначе False.
    """
    for neighbor in G.neighbors(vertex):
        if color_map[neighbor] == color:
            return False
    return True


def color_graph(vertex, num_colors, color_map, G):
    """
    Рекурсивно пытается окрасить граф, начиная с указанной вершины.

    :param vertex: Индекс вершины, с которой начинается окраска.
    :param num_colors: Количество доступных цветов.
    :param color_map: Текущее отображение вершин на цвета.
    :return: True, если граф успешно окрашен, иначе False.
    """
    if vertex == G.number_of_nodes():
        return True

    for color in range(num_colors):
        if is_safe(vertex, color, color_map, G):
            color_map[vertex] = color
            if color_graph(vertex + 1, num_colors, color_map, G):
                return True
            color_map[vertex] = -1

    return False


def calculate_chromatic_number(G):
    """
    Вычисляет хроматическое число графа с помощью алгоритма обратного отсчета.

    :return: Хроматическое число графа.
    """
    color_map = {node: -1 for node in G.nodes()}
    num_colors = 1

    while not color_graph(0, num_colors, color_map, G):
        num_colors += 1
        color_map = {node: -1 for node in G.nodes()}

    return num_colors

Работа с графами

In [ ]:
for i in range(1):
    num_nodes = random.randint(3, 5) # Количество узлов
    edge_probability = random.random() # Вероятность создания ребра
    G = generate_random_graph(num_nodes, edge_probability)
    filename = f"graph_{i}.png"
    save_graph(G, filename)
    print(f"Создан и сохранен граф: {filename}")

    diameter, radius = calculate_diameter_and_radius(G)
    print(f"Диаметр графа: {diameter}, Радиус графа: {radius}")

    max_degree, min_degree, mean_degree, median_degree, mode_value = meaning_of_degrees(G)
    print(f"Максимальная степень: {max_degree} \nМинимальная степень: {min_degree}\n"
          + f"Средняя степень: {mean_degree} \nМедианная степень: {median_degree}" +
          f"\nМода: {mode_value}")

    chromatic_number = calculate_chromatic_number(G)
    print(f"Хроматическое число графа: {chromatic_number}")


Создан и сохранен граф: graph_0.png
Диаметр графа: 1, Радиус графа: 1
Максимальная степень: 3 
Минимальная степень: 3
Средняя степень: 3.0 
Медианная степень: 3
Мода: 3
Хроматическое число графа: 4


Тесты

In [ ]:
G = nx.Graph()
G.add_nodes_from(range(5))
G.add_edge(0, 1)
G.add_edge(0, 2)
G.add_edge(1, 2)
G.add_edge(1, 3)
G.add_edge(2, 3)
G.add_edge(3, 4)
filename = f"graph_{1}.png"
save_graph(G, filename)
print(f"Создан и сохранен граф: {filename}")
degrees = [(node, val) for (node, val) in G.degree()]
print(degrees)

Создан и сохранен граф: graph_1.png
[(0, 2), (1, 3), (2, 3), (3, 3), (4, 1)]
